In [1]:
!nvidia-smi 

Thu Mar 10 12:13:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Global-Local\ Image-based\ Active\ Contour\ Loss\ for\ Multiclass\ Medical\ Image\ Segmentation\ with\ Deep\ Learning/ISIC
%run otherUnets.ipynb
%run preprocessing_2D.ipynb

In [17]:
!pip install pytorch-lightning
import pytorch_lightning as pl
from IPython.display import clear_output
import nibabel as nib
import csv
import os
import glob 
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
from fastprogress import master_bar, progress_bar
from torchvision import transforms
import torch.optim as optim
from itertools import product
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class HistoryLogger(pl.callbacks.Callback):
    def __init__(self, dir = "history_isicUnet_TSKY.csv"):
        self.dir = dir
    def on_validation_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        if "loss_epoch" in metrics.keys():
            logs = {"epoch": trainer.current_epoch}
            keys = ["loss_epoch", "train_dice_epoch", "train_jac_epoch",
                    "val_loss","val_dice", "val_jac"
                    ]
            for key in keys:
                logs[key] = metrics[key].item()
            header = list(logs.keys())
            isFile = os.path.isfile(self.dir)
            with open(self.dir, 'a', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=header)
                if not isFile:
                    writer.writeheader()
                writer.writerow(logs) 
        else:
            pass
def setDropProb(model, prob=0.01):
    for layer in model.modules():
        if isinstance(layer, DropBlock2D):
            layer.drop_prob = prob
clear_output()
############## turn off Debug APIs for Final Training############
torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

In [18]:
data = np.load("./dataISIC2018/ISIC2018_192_256.npz")
x,y = data["image"], data["mask"]
test_size = int((10/100)*x.shape[0])	
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = test_size, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = test_size, random_state=42)
del x,y, data, test_size

In [19]:
train_dataset = DataLoader(ISICLoader(x_train, y_train), batch_size=8, pin_memory=True,
                        shuffle=True, num_workers=2, 
                        drop_last=True, prefetch_factor = 16)
val_dataset = DataLoader(ISICLoader(x_val, y_val, typeData="test"), batch_size=16,
                          num_workers=2, prefetch_factor=16)
test_dataset = DataLoader(ISICLoader(x_test, y_test, typeData="test"), batch_size=16,
                          num_workers=2, prefetch_factor=16)

In [22]:
class U_Net(nn.Module):
    def __init__(self,img_ch=3,output_ch=2, drop_prob = 0):
        super(U_Net,self).__init__()
        
        self.Maxpool = nn.MaxPool2d(kernel_size=2,stride=2)
        channel = 64
        self.Conv1 = conv_block(ch_in=img_ch,ch_out=channel)
        self.Conv2 = conv_block(ch_in=channel,ch_out=channel*2)
        self.Conv3 = conv_block(ch_in=channel*2,ch_out=channel*4)
        self.Conv4 = conv_block(ch_in=channel*4,ch_out=channel*8, drop_block=True, block_size = 5, drop_prob = drop_prob)
        self.Conv5 = conv_block(ch_in=channel*8,ch_out=channel*16, drop_block=True, block_size = 3, drop_prob = drop_prob)

        self.Up5 = up_conv(ch_in=channel*16,ch_out=channel*8)
        self.Up_conv5 = conv_block(ch_in=channel*16, ch_out=channel*8)

        self.Up4 = up_conv(ch_in=channel*8,ch_out=channel*4)
        self.Up_conv4 = conv_block(ch_in=channel*8, ch_out=channel*4)
        
        self.Up3 = up_conv(ch_in=channel*4,ch_out=channel*2)
        self.Up_conv3 = conv_block(ch_in=channel*4, ch_out=channel*2)
        
        self.Up2 = up_conv(ch_in=channel*2,ch_out=channel)
        self.Up_conv2 = conv_block(ch_in=channel*2, ch_out=channel)

        self.Conv_1x1 = nn.Sequential(
            nn.Conv2d(channel, output_ch,kernel_size=1,stride=1,padding=0), 
            nn.Softmax(dim=1)
            )


    def forward(self,x):
        # encoding path
        x1 = self.Conv1(x)

        x2 = self.Maxpool(x1)
        x2 = self.Conv2(x2)
        
        x3 = self.Maxpool(x2)
        x3 = self.Conv3(x3)

        x4 = self.Maxpool(x3)
        x4 = self.Conv4(x4)

        x5 = self.Maxpool(x4)
        x5 = self.Conv5(x5)

        # decoding + concat path
        d5 = self.Up5(x5)
        d5 = torch.cat((x4,d5),dim=1)
        
        d5 = self.Up_conv5(d5)
        
        d4 = self.Up4(d5)
        d4 = torch.cat((x3,d4),dim=1)
        d4 = self.Up_conv4(d4)

        d3 = self.Up3(d4)
        d3 = torch.cat((x2,d3),dim=1)
        d3 = self.Up_conv3(d3)

        d2 = self.Up2(d3)
        d2 = torch.cat((x1,d2),dim=1)
        d2 = self.Up_conv2(d2)

        d1 = self.Conv_1x1(d2)

        return d1


In [23]:
class TverskyLoss(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.device = device
    def forward(self, y_true, y_pred):   
        yTrueOnehot = torch.zeros(y_true.size(0), NUM_CLASS, y_true.size(2), y_true.size(3), device=self.device)
        yTrueOnehot = torch.scatter(yTrueOnehot, 1, y_true, 1)[:, 1:]
        y_pred = y_pred[:,1:]
        TP = torch.sum(yTrueOnehot * y_pred, dim=[1,2,3])
        FN = torch.sum(yTrueOnehot * (1-y_pred), dim=[1,2,3])
        FP = torch.sum((1-yTrueOnehot) * y_pred, dim=[1,2,3])
        loss = torch.mean((0.7 * FP + 0.3 * FN)/(TP + 0.7 * FP + 0.3 * FN + 1e-8))
        return loss

In [24]:
class Segmentor(pl.LightningModule):
    def __init__(self, model = U_Net(drop_prob=0)):
        super().__init__()
        self.model = model
    def forward(self, x):
        return self.model(x)
    def get_metrics(self):
        # don't show the version number
        items = super().get_metrics()
        items.pop("v_num", None)
        return items

    # def _step(self, batch):
    #     image, y_true = batch
    #     y_pred = self.model(image)
    #     loss = SemiActiveLoss(device=self.device)(image, y_true, y_pred)
    #     dice_score, jaccard_score = dice(y_true, y_pred), jaccard(y_true, y_pred)
    #     return loss, dice_score, jaccard_score
    def _step(self, batch):
        image, y_true = batch
        y_val = y_true[:,0]
        y_pred = self.model(image)
        loss = TverskyLoss(device=self.device)(y_true, y_pred)
        dice_score, jaccard_score = dice(y_true, y_pred), jaccard(y_true, y_pred)
        return loss, dice_score, jaccard_score

    def training_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"loss": loss, "train_dice": dice_score, "train_jac": jaccard_score}
        self.log_dict(metrics, on_step=True, on_epoch=True, prog_bar = True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"val_loss": loss, "val_dice": dice_score, "val_jac":jaccard_score}
        self.log_dict(metrics, prog_bar = True)
        return metrics

    def test_step(self, batch, batch_idx):
        loss, dice_score, jaccard_score = self._step(batch)
        metrics = {"test_loss": loss, "test_dice": dice_score, "test_jac":jaccard_score}
        self.log_dict(metrics, prog_bar = True)
        return metrics
        

    def configure_optimizers(self):
        optimizer = Nadam(self.parameters(), lr=1e-3)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max",
                                                         factor = 0.5, patience=15, verbose =True)
        lr_schedulers = {"scheduler": scheduler, "monitor": "val_dice"}
        return [optimizer], lr_schedulers
    

In [29]:
segmentor = Segmentor(U_Net(drop_prob=0))
check_point = pl.callbacks.model_checkpoint.ModelCheckpoint("./weightUnet/", filename="ckpt{val_dice:0.4f}",
                                                            monitor="val_dice", mode = "max", save_top_k =1,
                                                            verbose=True, save_weights_only=True,
                                                            auto_insert_metric_name=False,)
progress_bar = pl.callbacks.TQDMProgressBar()
logger = HistoryLogger()
swa = pl.callbacks.StochasticWeightAveraging(swa_epoch_start=25)
PARAMS = {"gpus":1, "benchmark": True, "enable_progress_bar" : False, "logger":False,
        #   "callbacks" : [progress_bar], 
        #    "overfit_batches" :1, 
          "callbacks" : [check_point, progress_bar, logger], 
          "log_every_n_steps" :1, "num_sanity_val_steps":1, "max_epochs":30,
          "precision":16,
          }

trainer = pl.Trainer(**PARAMS)
# segmentor = Segmentor.load_from_checkpoint(checkpoint_path="./weightUnet/current.ckpt") 
segmentor = Segmentor.load_from_checkpoint(checkpoint_path="./weightUnet/ckpt0.7812.ckpt")


INFO:pytorch_lightning.utilities.distributed:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(segmentor, train_dataset, val_dataset)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type  | Params
--------------------------------
0 | model | U_Net | 31.0 M
--------------------------------
31.0 M    Trainable params
0         Non-trainable params
31.0 M    Total params
62.077    Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /content/drive/MyDrive/Global-Local Image-based Active Contour Loss for Multiclass Medical Image Segmentation with Deep Learning/ISIC/weightUnet exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.distributed:Epoch 0, global step 258: val_dice reached 0.76773 (best 0.76773), saving model to "/content/drive/MyDrive/Global-Local Image-based Active Contour Loss for Multiclass Medical Image Segmentation with Deep Learning/ISIC/weightUnet/ckpt0.7677.ckpt" as top 1


...

INFO:pytorch_lightning.utilities.distributed:Epoch 5, global step 1553: val_dice reached 0.79677 (best 0.79677), saving model to "/content/drive/MyDrive/Global-Local Image-based Active Contour Loss for Multiclass Medical Image Segmentation with Deep Learning/ISIC/weightUnet/ckpt0.7968.ckpt" as top 1


Validating: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.distributed:Epoch 6, global step 1812: val_dice was not in top 1


Validating: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.distributed:Epoch 7, global step 2071: val_dice was not in top 1


In [ ]:
# for drop_prob in np.linspace(0.01, 0.1, 10):
#     setDropProb(segmentor, drop_prob) 
#     trainer = pl.Trainer(**PARAMS)
#     trainer.fit(segmentor, train_dataset, val_dataset)

In [ ]:
trainer.save_checkpoint("./weightUnet/current.ckpt", weights_only=True)

In [30]:
for layer in segmentor.modules():
    if isinstance(layer, DropBlock2D):
        layer.drop_prob = 0.1
        
for layer in segmentor.modules():
    if isinstance(layer, DropBlock2D):
        print(layer.drop_prob) 

# segmentor.configure_optimizers()

0.1
0.1


In [ ]:
trainer.test(segmentor, test_dataset)

In [ ]:
weight_path = sorted(glob.glob("./weightUnet/*"), reverse=True)
max_score = 0
best_weight = ""
for weight in tqdm(weight_path[2:]):
    segmentor = Segmentor.load_from_checkpoint(checkpoint_path=weight)
    dice_score = trainer.test(segmentor, test_dataset)[0]["test_dice"]
    # clear_output()
    print(dice_score)
    if max_score < dice_score:
        max_score = dice_score
        best_weight = weight
print(f"best weight is: {best_weight} with {max_score}")


In [ ]:
def predict(images, model, batch_size = 64):
    images = torch.as_tensor(images, dtype= torch.float32)
    y_preds = torch.zeros((images.size(0), NUM_CLASS, images.size(2), images.size(3)), device= device)
    batch_start = 0
    batch_end = batch_size
    pbar = tqdm()
    while batch_start < images.size(0):
        image = images[batch_start : batch_end]
        with torch.inference_mode():
            image = image.to(device)
            y_pred = model(image)
            y_preds[batch_start : batch_end] = y_pred
        batch_start += batch_size
        batch_end += batch_size
        pbar.update(1)
    pbar.close()
    res = y_preds.cpu().numpy()
    del y_preds
    return res

In [ ]:
weight_path = sorted(glob.glob("./weightUnet/*"), reverse=True)
weight_path 

In [ ]:
from sklearn.metrics import recall_score, precision_score, precision_recall_fscore_support
x_inf = x_test.transpose(0, -1, 1, 2) / 255
segmentor = Segmentor(U_Net(drop_prob=0))
segmentor = Segmentor.load_from_checkpoint(checkpoint_path="./weightUnet/ckpt0.8680.ckpt")
segmentor = segmentor.to(device)
segmentor.eval()
y_pred = predict(x_inf, segmentor)
torch.cuda.empty_cache()
mask_predict = np.argmax(y_pred, axis=1)


In [ ]:
a = y_test.reshape(y_test.shape[0], -1)
b = mask_predict.reshape(mask_predict.shape[0], -1)
precision_recall_fscore_support(a, b, average="samples")

(0.8866047180831157, 0.9088718583461524, 0.8808378579339233, None)

In [ ]:
inter = np.sum(a * b, axis=1)
uni = np.sum(a + b,  axis=1)
dice1 = (2*inter+1e-5) / (uni+1e-5) 

uni1 = np.sum(a + b - a*b,  axis=1)
jac = (inter+1e-5) / (uni1+1e-5) 
print(np.mean(dice1), np.mean(jac))

0.8808378581183421 0.8062207570242205


In [ ]:
# %cd /content/drive/MyDrive/new_paper/ISIC/dataISIC2018
np.savez_compressed("ISIC2018_Unet_CE", predicted_mask=mask_predict)

In [ ]:
for i in range(x_test.shape[0]):
    plt.figure(i+1)
    plt.subplot(131), plt.imshow(x_test[i])
    plt.subplot(132), plt.imshow(y_test[i])
    plt.subplot(133), plt.imshow(mask_predict[i])
